In [1]:
# File needs to be run twice, once for swimdata and second for swimdata2

import DBcm
import os
import swim_utils
import datetime

print("Enter the absolute path of the folder to ingest: ")
swim_utils.FOLDER = input() # input the absolute path of the folder to add and add "/" to the end. e.g /home/adamfitz395/Cloud/WebDev/CA/03/uploaded/swimdata/

files = os.listdir(swim_utils.FOLDER)

files.remove(".DS_Store")
swimmer_data = []
event_data = []
time_data = []

config = {
    "user": "swimuser",
    "password": "swimuserpasswd",
    "database": "swimmer_database",
    "host": "localhost",
}

with open("ingest_data.ipynb") as f:
    data = f.read()

Enter the absolute path of the folder to ingest: 


In [2]:
#Populate swimmers_data
for file in files:
    print(file)
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file)
    if (name, age) in swimmer_data:
        continue
    else:
        swimmer_data.append((name,age))
        
# Modify the SQL query
swim_insert_SQL = """    
    INSERT INTO swimmers
    (name, age)
    SELECT %s, %s
    WHERE NOT EXISTS (
        SELECT 1
        FROM swimmers
        WHERE name = %s AND age = %s
    );
"""

# Modify the execute call
with DBcm.UseDatabase(config) as db:
    for name, age in swimmer_data:
        db.execute(swim_insert_SQL, (name, age, name, age))

Blake-15-100m-Fly.txt
Darius-13-100m-Fly.txt
Maria-9-50m-Free.txt
Owen-15-100m-Free.txt
Darius-13-100m-Back.txt
Darius-13-200m-IM.txt
Katie-9-50m-Back.txt
Katie-9-100m-Breast.txt
Katie-9-50m-Free.txt
Katie-9-100m-IM.txt
Katie-9-100m-Back.txt
Blake-15-100m-Back.txt
Katie-9-50m-Fly.txt
Darius-8-50m-Breast.txt
Abi-10-100m-Breast.txt
Darius-13-100m-Breast.txt
Katie-9-100m-Free.txt
Dave-17-100m-Free.txt
Darius-8-50m-Free.txt
Abi-10-100m-Back.txt
Dave-17-200m-Back.txt
Darius-8-50m-Back.txt
Katie-9-50m-Breast.txt
Blake-15-100m-Free.txt


In [3]:
# Populate event_data        
for file in files:
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file)
    if (distance, stroke) in event_data:
        continue
    else:
        event_data.append((distance,stroke))
        
event_insert_SQL = """
    INSERT INTO events 
    (distance, event)
    SELECT %s, %s
    WHERE NOT EXISTS (
        SELECT 1
        FROM events
        WHERE distance = %s AND event = %s
    );
"""

# Modify the execute call for events
with DBcm.UseDatabase(config) as db:
    for distance, stroke in event_data:
        db.execute(event_insert_SQL, (distance, stroke, distance, stroke))

In [4]:
swim_select_SQL = """
    SELECT swimmer_id 
    FROM swimmers 
    WHERE name = %s AND age = %s;"""

swimmer_ids = []

for swimmer in files:
    name, age = swim_utils.get_swimmers_data(swimmer)[:2]

    with DBcm.UseDatabase(config) as db:
        db.execute(swim_select_SQL, (name, age))
        swimmer_ids.append(db.fetchone())

print(swimmer_ids)

[(4,), (6,), (7,), (12,), (6,), (6,), (18,), (18,), (18,), (18,), (18,), (4,), (18,), (23,), (9,), (6,), (18,), (22,), (23,), (9,), (22,), (23,), (18,), (4,)]


In [5]:
event_select_SQL = """
    SELECT event_id 
    FROM events 
    WHERE distance = %s AND event = %s;"""
    
event_ids = []
for event in files:
    
    distance, stroke = swim_utils.get_swimmers_data(event)[2:4]
    
    with DBcm.UseDatabase(config) as db:
        db.execute(event_select_SQL, (distance, stroke))
        event_ids.append(db.fetchone())
            
print(event_ids)

[(1,), (1,), (2,), (9,), (10,), (11,), (3,), (4,), (2,), (14,), (10,), (10,), (7,), (6,), (4,), (4,), (9,), (9,), (2,), (10,), (5,), (3,), (6,), (9,)]


In [6]:
ts = datetime.datetime.now()

print(ts)

current = 0
for file in files:
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file)

    with DBcm.UseDatabase(config) as db:
        for time in the_times:
            insert_times_SQL = """
            INSERT INTO times (swimmer_id, event_id, times, ts) 
            VALUES (%s, %s, %s, %s);
            """
            db.execute(insert_times_SQL, (swimmer_ids[current][0], event_ids[current][0], time, ts))
    current += 1
        
print(swimmer_data)
print(event_data)

2023-12-11 13:03:22.694583
[('Blake', '15'), ('Darius', '13'), ('Maria', '9'), ('Owen', '15'), ('Katie', '9'), ('Darius', '8'), ('Abi', '10'), ('Dave', '17')]
[('100m', 'Fly'), ('50m', 'Free'), ('100m', 'Free'), ('100m', 'Back'), ('200m', 'IM'), ('50m', 'Back'), ('100m', 'Breast'), ('100m', 'IM'), ('50m', 'Fly'), ('50m', 'Breast'), ('200m', 'Back')]
